# Clase Nº 6

**Plan de la clase:**  
**(1)** Preguntas sobre la clase asincrónica. <br>
**(2)** Repaso de `pd.DataFrame.merge`. <br>
**(3)** Unir un dataframe con un geodataframe y visualizar atributos en un mapa <br>
**(4)** Unir dos geodataframes en base a atributos espaciales <br>

### Preguntas sobre la clase asincrónica

...

### Repaso de `pd.DataFrame.merge`

### Unir un geodataframe con un dataframe a través de un atributo no espacial

Vamos a ejemplificar cómo unir un `DataFrame` de Pandas con un `GeoDataFrame` de Geopandas.
Para eso vamos a utilizar una tabla que contiene la población de las distintas provincias argentinas, y el shapefile de las provincias que ya vimos en la clase asincrónica.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

Descarguemos la tabla de población por provincia de [esta página](https://www.ign.gob.ar/NuestrasActividades/Geografia/DatosArgentina/Poblacion2) del sitio del IGN.
Para eso utilizamos la función `read_html` de `Pandas` que es capaz de extraer tablas de un archivo `html`.

In [ ]:
url_pob = "https://www.ign.gob.ar/NuestrasActividades/Geografia/DatosArgentina/Poblacion2"
tablas = pd.read_html(url_pob)

In [ ]:
poblacion_df = tablas[2]

In [ ]:
poblacion_df

Preprocesamos el dataframe de población:

In [ ]:
def convertir_a_int(fila, cols=["Año 2010", "Año 2015", "Año 2020"]):    

    '''
    Convertir cols, que contienen strings de números con "." como separador de miles, a tipo entero.
    '''
    
    try:
        for col in cols:
            fila[col] = int(fila[col].replace(".",""))
    except:
        pass
        
    return fila

In [ ]:
poblacion_df = poblacion_df.apply(convertir_a_int, axis=1)

In [ ]:
provincias_gdf = gpd.read_file(filename="datos/provincia.zip", bbox=(-75,-50,-55,-20))
provincias_gdf = provincias_gdf[["nam", "geometry"]]
provincias_gdf

### Ejercicio

Unir las bases `provincias_gdf` y `poblacion_df` por el nombre de la provincia. Notar que necesitamos especificar `left_on` y `right_on` porque los nombres de las columnas correspondientes son diferentes (o bien cambiar el nombre de una de las columnas).
A continuación, representar un mapa del país coloreando cada provincia en base a su población.

### Unir geodatraframes por un atributo espacial: `sjoin`

Geopandas permite unir dos GeoDataFrames en base a relaciones espaciales entre las observaciones de cada uno.

Vamos a usar estas relaciones espaciales para poder vincular departamentos con provincias.

In [ ]:
departamentos_gdf = gpd.read_file("datos/departamento.zip")
departamentos_gdf = departamentos_gdf[["objectid", "nam", "geometry"]]
departamentos_gdf = departamentos_gdf.rename({"nam":"departamento"}, axis=1)

Obtenemos los centroides de cada departamento para establecer la relación de pertenencia con la provincia.

In [ ]:
departamentos_centroid_gdf = departamentos_gdf.copy()
departamentos_centroid_gdf["geometry"] = departamentos_gdf.geometry.centroid;

In [ ]:
prov_dep_gdf = departamentos_centroid_gdf.sjoin(provincias_gdf, predicate="within")

In [ ]:
prov_dep_df = prov_dep_gdf[["objectid", "provincia"]]

In [ ]:
departamentos_gdf = departamentos_gdf.merge(prov_dep_df, how="inner", on="objectid")

Ahora representamos la división departamental de cada provincia en un mapa.

In [ ]:
def mostrar_deptos_por_provincia(provincia):
    tmp_gdf = departamentos_gdf[departamentos_gdf["provincia"]==provincia]
    tmp_gdf.plot(cmap="tab10", figsize=(10,10))

In [ ]:
import ipywidgets as widgets 
from ipywidgets import interact

In [ ]:
interact(
    mostrar_deptos_por_provincia, 
    provincia=widgets.Dropdown(options=sorted(pd.unique(provincias.nam)))
);